In [1]:
%pip install pyradiomics dicom_numpy pydicom plotly matplotlib scikit-image simpleITK pynrrd dicom2nifti NiBabel NiLearn openpyxl pydicom-seg pandarallel

Note: you may need to restart the kernel to use updated packages.


# Pyradiomics Extraction

Note: if you need to save somewhere else, please make chanegs to the EXTRACT_NAME variable. For reading different segmentation labels, make changes to the get_features function.

Assumes that the segmentation files are placed right beside the pre and post_1 sequences in the dataset.

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import dicom_numpy
import pydicom as dicom

import dicom2nifti
import nibabel as nib
import nilearn as nil
import scipy.ndimage as ndi
import SimpleITK as sitk
import os

from radiomics import featureextractor

from tqdm import tqdm

from pandarallel import pandarallel

import re

tqdm.pandas()
pandarallel.initialize(progress_bar=True,nb_workers= 40)

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [3]:
annotation_boxes = pd.read_csv("../Data/segmentation_annotations_NIFTI.csv")#.set_index("Patient_ID")

DATASET_PATH = '../../Processed NIFTI Dataset/'

EXTRACT_NAME = "../Data/pyradiomics_extraction.csv"

In [7]:
feat_ext = featureextractor.RadiomicsFeatureExtractor()
feat_ext.settings['minimumROIDimensions'] = 2
feat_ext.settings['normalize'] = True
feat_ext.settings['additionalInfo'] = False
feat_ext.settings['correctMask'] = True
feat_ext.settings

{'minimumROIDimensions': 2,
 'minimumROISize': None,
 'normalize': True,
 'normalizeScale': 1,
 'removeOutliers': None,
 'resampledPixelSpacing': None,
 'interpolator': 'sitkBSpline',
 'preCrop': False,
 'padDistance': 5,
 'distances': [1],
 'force2D': False,
 'force2Ddimension': 0,
 'resegmentRange': None,
 'label': 1,
 'additionalInfo': False,
 'correctMask': True}

In [14]:
def get_features(x):
    from radiomics import featureextractor
    import SimpleITK as sitk
    import os
    import numpy as np
    SEGMENTATION_FILENAME = 'segmentation.img.gz'

    feat_ext = featureextractor.RadiomicsFeatureExtractor()
    feat_ext.settings['minimumROIDimensions'] = 2
    feat_ext.settings['normalize'] = True
    feat_ext.settings['additionalInfo'] = False
    feat_ext.settings['correctMask'] = True
    feat_ext.settings

    filenames = ['pre.img.gz', 'post_1.img.gz']
    DATASET_PATH = '../../Processed NIFTI Dataset/'

    
    patient_id = x['Patient_ID']
    file_path = os.path.join(DATASET_PATH, patient_id, 'post_1.img.gz')                # only for post1
    segmentation_path = os.path.join(DATASET_PATH, patient_id, SEGMENTATION_FILENAME)  # segmentation

    img = sitk.ReadImage(file_path)
    smt = sitk.ReadImage(segmentation_path)

    smt.SetOrigin(img.GetOrigin())

    feats = feat_ext.execute(img, smt)
    feats['patient'] = patient_id
    feats['sequence'] = 'post_1'

    return feats

In [15]:
pyradiomics_features = annotation_boxes.parallel_apply(get_features, axis = 1)

In [24]:
myfeatures = pyradiomics_features.tolist()
feature_df = pd.DataFrame()

for i in myfeatures:
    temp = {}
    for key, value in i.items():
        if(isinstance(value, str)):
            temp[key] = value
        else:
            temp[key] = value.item()

    ft_df = pd.DataFrame(temp, index = [0])
    feature_df = pd.concat([feature_df, ft_df])

In [28]:
feature_df = feature_df.set_index(['patient', 'sequence']).reset_index()

feature_df

patient sequence  original_shape_Elongation  \
0    Breast_MRI_001   post_1                   0.807005   
1    Breast_MRI_002   post_1                   0.641558   
2    Breast_MRI_003   post_1                   0.756223   
3    Breast_MRI_004   post_1                   0.718139   
4    Breast_MRI_005   post_1                   0.687760   
..              ...      ...                        ...   
917  Breast_MRI_918   post_1                   0.886914   
918  Breast_MRI_919   post_1                   0.757419   
919  Breast_MRI_920   post_1                   0.872632   
920  Breast_MRI_921   post_1                   0.855874   
921  Breast_MRI_922   post_1                   0.896751   

     original_shape_Flatness  original_shape_LeastAxisLength  \
0                   0.729780                       23.614309   
1                   0.577887                       17.378564   
2                   0.272616                       13.050754   
3                   0.610317                        8.730117   
4                   0.436019                       22.539193   
..                       ...                             ...   
917                 0.668151                       28.553043   
918                 0.596730                       12.864613   
919                 0.837806                       15.199560   
920                 0.735409                       30.453436   
921                 0.767845                        9.406096   

     original_shape_MajorAxisLength  original_shape_Maximum2DDiameterColumn  \
0                         32.358102                               36.073737   
1                         30.072592                               23.148662   
2                         47.872295                               40.488107   
3                         14.304230                               11.420813   
4                         51.693090                               55.175581   
..                              ...                                     ...   
917                       42.734408                               45.886501   
918                       21.558522                               21.066007   
919                       18.142097                               19.038279   
920                       41.410193                               43.076382   
921                       12.249994                               14.031840   

     original_shape_Maximum2DDiameterRow  \
0                              38.431136   
1                              29.852205   
2                              43.126056   
3                              14.972284   
4                              49.674566   
..                                   ...   
917                            41.977361   
918                            20.543932   
919                            20.649190   
920                            45.448460   
921                            12.641847   

     original_shape_Maximum2DDiameterSlice  original_shape_Maximum3DDiameter  \
0                                39.309346                         45.798426   
1                                29.748492                         33.784677   
2                                56.606869                         57.665740   
3                                14.477716                         16.540987   
4                                36.443449                         58.274413   
..                                     ...                               ...   
917                              51.383808                         56.016573   
918                              25.375225                         27.268354   
919                              20.505122                         24.348411   
920                              48.143552                         54.398662   
921                              13.078416                         16.022136   

     ...  original_glszm_SmallAreaHighGrayLevelEmphasis  \
0    ...                            

In [29]:
feature_df.to_csv(EXTRACT_NAME, index = False)